In [1]:
import numpy as np
import pyxdf
import logging
import json

# Configure logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s.%(msecs)04d %(levelname)s: %(message)s',
    datefmt='%Y-%m-%d %H:%M:%S'
)
logger = logging.getLogger(__name__)

FRONTAL_CHANNELS      = np.array(['Fp1', 'Fp2', 'F7', 'F3', 'Fz', 'F4', 'F8'])
CENTRAL_L_CHANNELS    = np.array(['T3', 'C3', 'P3', 'T5'])
CENTRAL_R_CHANNELS    = np.array(['T4', 'C4', 'P4', 'T6'])
OCCIPITAL_CHANNELS    = np.array(['O1', 'O2'])
MIDLINE_UNUSED        = np.array(['Cz', 'Pz'])

CHANNELS_IN_STREAM = np.array(['Fp1', 'Fp2', 'F3', 'F4', 'C3', 'C4', 'P3', 'P4', 'O1', 'O2', 'F7', 'F8', 'T3', 'T4', 'T5', 'T6', 'A1', 'A2', 'Fz', 'Cz', 'Pz'])

In [5]:
aa = np.array([{'s':1},{'s':2}])

aa[1]['s']

2

In [2]:
xdf_file = '/Users/vladimirantipov/Documents/Brainbuilding/database/2025/09/248cf8aa-f5b3-4e0d-81e2-816bdeb98670/data.xdf'#'/Users/vladimirantipov/Documents/Neurolab/Experiments/HandMoves/dataset3/data/3/data.xdf'
json_file = '/Users/vladimirantipov/Documents/Brainbuilding/database/2025/09/248cf8aa-f5b3-4e0d-81e2-816bdeb98670/Task.json'#'/Users/vladimirantipov/Documents/Neurolab/Experiments/HandMoves/dataset3/data/3/Task.json'

with open(json_file, encoding='utf-8') as file:
    task = json.load(file)

patient = task['patient']

streams, _ = pyxdf.load_xdf(xdf_file, synchronize_clocks=True, dejitter_timestamps=True)
streams = {stream["info"]["stream_id"]: stream for stream in streams}

2025-09-06 17:21:21.0008 INFO: Importing XDF file /Users/vladimirantipov/Documents/Brainbuilding/database/2025/09/248cf8aa-f5b3-4e0d-81e2-816bdeb98670/data.xdf...
2025-09-06 17:21:21.0102 INFO:   performing clock synchronization...
2025-09-06 17:21:21.0130 INFO:   performing jitter removal...


In [25]:
name = ' '.join([patient['lastName'], patient['firstName'],  patient['middleName']])

from datetime import datetime
dt = datetime.strptime(task['StartTime'], '%d.%m.%Y %H:%M:%S')
date_str = dt.strftime('%d.%m.%Y')
print(task['StartTime'].split(' ')[0])

23.12.2024


In [26]:
def _get_stream_id(streams, stream_name=None, stream_type=None):
    """Find stream ID by name and/or stream type."""
    for stream_id, stream in streams.items():
        info = stream['info']
        if stream_name is not None and info['name'][0] != stream_name:
            continue
        if stream_type is not None and info['type'][0] != stream_type:
            continue
        return stream_id
    criteria = []
    if stream_name: criteria.append(f"name='{stream_name}'")
    if stream_type: criteria.append(f"type='{stream_type}'")
    logger.error(f"Stream with {' and '.join(criteria)} not found")
    return None

egg_id = _get_stream_id(streams, stream_type='EEG')
eeg_stream = streams[egg_id]

ch_list, units = [], []
for ch in eeg_stream["info"]["desc"][0]["channels"][0]["channel"]:
    ch_list.append(str(ch["label"][0]))
    units.append(ch["unit"][0] if ch["unit"] else "NA")

time_series = eeg_stream["time_series"]
microvolts = ("microvolt", "microvolts", "µV", "μV", "uV")
scale = np.array([1e-6 if u in microvolts else 1 for u in units])
time_series_scaled = (time_series * scale).T

def _get_events(task):
    events = task['events']
    samples = task['samples']

    event_dict = {'source': [],'event_name': [],'sample_type': [],'trial_type': [],'block_type': [],
    'block_id': [],'trial_id': [],'event_id': [],'item_id': [],'sample_id': [],'trigger_code': [],
    'time': [],'index': [],'duration': [] }

    for event in events:
        sample = samples[event['sample_id']]
        event_dict['source'].append(event['source'])
        event_dict['event_name'].append(event['event_name'])
        event_dict['sample_type'].append(sample['sample_type'])
        event_dict['trial_type'].append(sample['trial_type'])
        event_dict['block_type'].append(sample['block_type'])
        event_dict['block_id'].append(sample['block_id'])
        event_dict['trial_id'].append(sample['trial_id'])
        event_dict['event_id'].append(event['event_id'])
        event_dict['item_id'].append(event['item_id'])
        event_dict['sample_id'].append(event['sample_id'])
        event_dict['trigger_code'].append(sample['trigger_code'])
        event_dict['time'].append(event['time'])
        event_dict['duration'].append(float(sample['duration']) if event['event_name'] == 'show' else 0.0)
    for key in event_dict:
        event_dict[key] = np.array(event_dict[key])
    return event_dict

events = _get_events(task)


# ERD

In [27]:
def rest_stim_bands(events, hand, first_time, srate):
    rest = np.where(
        (events['sample_type'] == 'Rest') &
        (events['trial_type'] == hand+'/hand') &
        (events['event_name'] == 'show')
    )[0]
    rest_time = events['time'][rest] - first_time
    stim = np.where(
        (np.isin(events['sample_type'], ['Point', 'Image'])) &
        (events['trial_type'] == hand+'/hand') &
        (events['event_name'] == 'show')
    )[0]
    stim_time = events['time'][stim] - first_time

    rest_start = np.round(rest_time * srate).astype(int)
    rest_end = np.round((rest_time+events['duration'][rest]) * srate).astype(int)
    rest = np.column_stack((rest_start, rest_end))

    stim_start = np.round(stim_time * srate).astype(int)
    stim_end = np.round((stim_time+events['duration'][stim]) * srate).astype(int)
    stim = np.column_stack((stim_start, stim_end))
    return rest, stim

first_time = eeg_stream["time_stamps"][0]
srate = int(float(eeg_stream["info"]["nominal_srate"][0]))
rest_l, stim_l = rest_stim_bands(events, 'left', first_time, srate)
rest_r, stim_r = rest_stim_bands(events, 'right', first_time, srate)

data_f = bandpass_filter(time_series_scaled, 500, l_freq=1, h_freq=40, method='iir')

def calc_trials(rest, stim):
    r"""
    Returns
    -------
    trials : ndarray
        (trials_n, f_bands, ch_n)
    """

    trials = []
    for (r_start, r_end), (s_start, s_end) in zip(rest, stim):
        rest_data = data_f[:, r_start:r_end]
        stim_data = data_f[:, s_start:s_end]

        rest_psds, rest_freqs = psd_welch(rest_data, sfreq=srate, fmin=1, fmax=30)
        stim_psds, stim_freqs = psd_welch(stim_data, sfreq=srate, fmin=1, fmax=30)

        def power_band(psds, freqs, bands):
            band_powers = {}
            for band, (fmin, fmax) in bands.items():
                mask = (freqs >= fmin) & (freqs < fmax)
                band_powers[band] = np.mean(psds[:, mask], axis=1)
            return band_powers

        bands = {'mu': (8, 13),'beta': (13, 30)}
        rest_bands = power_band(rest_psds, rest_freqs, bands)
        stim_bands = power_band(stim_psds, stim_freqs, bands)

        ERD_mu = (rest_bands['mu'] - stim_bands['mu']) / rest_bands['mu'] * 100
        ERD_beta = (rest_bands['beta'] - stim_bands['beta']) / rest_bands['beta'] * 100
        ERD_mean = (ERD_mu + ERD_beta) / 2
        trials.append([ERD_mu, ERD_beta, ERD_mean])
    return np.array(trials)

l_trials = calc_trials(rest_l, stim_l)
r_trials = calc_trials(rest_r, stim_r)

# Make Json
result = {}
for hand, data in [('left', l_trials), ('right', r_trials)]:
    ch_dict = {}
    for ch_name, ch_idx in [('c3', 1)]:
        ch_data = data[:,:,ch_idx]
        band_dict = {}
        for band, band_idx in (('mu', 0), ('beta', 1) , ('erd', 2)):
            band_data = ch_data[:, band_idx]
            band_dict[band] = {'mean': np.mean(band_data), 'max': np.max(band_data)}
        ch_dict[ch_name] = band_dict
    result[hand] = ch_dict

out = {}
for hand, data in result.items():
    LI_ERD_mu   = ((data['c3']['mu']['mean']-data['c4']['mu']['mean']) /
                  (data['c3']['mu']['mean']+data['c4']['mu']['mean']))
    LI_ERD_beta = ((data['c3']['beta']['mean']-data['c4']['beta']['mean']) /
                  (data['c3']['beta']['mean']+data['c4']['beta']['mean']))
    LI_ERD_mean = ((data['c3']['erd']['mean']-data['c4']['erd']['mean']) /
                  (data['c3']['erd']['mean']+data['c4']['erd']['mean']))
    out[hand] = {'mu': LI_ERD_mu, 'beta': LI_ERD_beta, 'erd': LI_ERD_mean}

NameError: name 'bandpass_filter' is not defined

In [20]:
def truncate(val, decimals=3):
    sig = max(0, 4 - len(str(int(val))))
    return float(format(val , '.%df' % sig))
# def truncate(val, sig_digits=4):
#     v = float(val)
#     sig = max(0, sig_digits - len(str(int(abs(v)))) )  # сколько оставить после запятой
#     return round(v, sig)

value = 0.3452324
print(truncate(value, 3))

0.345


# background

In [4]:
first_time = eeg_stream["time_stamps"][0]
background_idxs = np.where(events['sample_type'] == 'Background')[0]
background_time = events['time'][background_idxs] - first_time
dur = events['duration'][background_idxs]

srate = int(float(eeg_stream["info"]["nominal_srate"][0]))
background_start = np.round(background_time * srate).astype(int)
background_end = np.round((background_time+dur) * srate).astype(int)

print(background_start, background_end, background_end-background_start)

background_data = []
for start, end in zip(background_start, background_end):
    background_data.append(time_series_scaled[:, start:end])

[ 32503 513540] [ 62503 543540] [30000 30000]


In [29]:
from scipy.signal import firwin, butter, filtfilt, sosfiltfilt

def bandpass_filter(data, sfreq, l_freq, h_freq, method='fir'):
    if method == 'fir':
        l_trans = min(max(0.25 * l_freq, 2.0), l_freq)
        h_trans = min(max(0.25 * h_freq, 2.0), sfreq/2 - h_freq)
        trans_bandwidth = min(l_trans, h_trans)
        n_taps = int(3.3 * sfreq / trans_bandwidth) | 1
        filt = firwin(n_taps, [l_freq, h_freq], fs=sfreq, pass_zero='bandpass')
        return filtfilt(filt, 1, data, axis=-1)
    else:
        nyq = sfreq / 2
        sos = butter(4, [l_freq/nyq, h_freq/nyq], 'band', output='sos')
        return sosfiltfilt(sos, data, axis=-1)

d1 = bandpass_filter(time_series_scaled, 500, l_freq=1, h_freq=40, method='fir')
d2 = bandpass_filter(time_series_scaled, 500, l_freq=1, h_freq=40, method='iir')


In [37]:
#6.377 0.758 1.346 0.174 0.338 0.052 0.11 0.017

sum = 6.377 + 1.346 + 0.338 + 0.11
print(np.sum([6.377/sum, 1.346/sum, 0.338/sum, 0.11/sum]))

1.0


In [30]:
from scipy.signal import welch
import mne

def psd_welch(data, sfreq, fmin=0, fmax=np.inf):
    nperseg = int(min(2 * sfreq, data.shape[-1]))
    nfft = max(nperseg, int(4 * sfreq))
    freqs, psds = welch(data, sfreq, nperseg=nperseg,nfft=nfft, noverlap=nperseg//2)
    freq_mask = (freqs >= fmin) & (freqs <= fmax)
    psds = psds[:, freq_mask] if psds.ndim > 1 else psds[freq_mask]
    return psds, freqs[freq_mask]

def power_band(psds, freqs, bands, power='abs'):
    psd = psds.copy()
    if power == 'rel':
        psd /= np.sum(psd, axis=-1, keepdims=True)
    band_powers = {}
    for band, (fmin, fmax) in bands.items():
        mask = (freqs >= fmin) & (freqs < fmax)
        band_powers[band] = np.mean(psd[:, mask], axis=1)
    return band_powers


bands = {'delta': (0.5, 4),'theta': (4, 8),'alpha': (8, 13),'beta': (13, 30)}
psds, freqs = psd_welch(d1, sfreq=500.0, fmin=1, fmax=30)
band_power_abs = power_band(psds, freqs, bands)
#band_powers = power_band(psds, freqs, bands, power='rel')

band_power_rel = {
    band: power / sum(band_power_abs.values())
    for band, power in band_power_abs.items()
}
#band_power_rel = np.array(np.array(list(band_power_abs.values())))

for ch in range(len(band_power_abs['delta'])):
   ch_psd = (band_power_abs['delta'][ch] + band_power_abs['theta'][ch] + band_power_abs['alpha'][ch] + band_power_abs['beta'][ch])

######### MNE TEST ##################
#fdata = mne.filter.filter_data(time_series_scaled, sfreq=500.0, l_freq=1, h_freq=30.0)
#psds2, freqs2 = mne.time_frequency.psd_array_welch(fdata, sfreq=500.0, fmin=0.5, fmax=30)


import matplotlib.pyplot as plt
%matplotlib qt6

plt.plot(freqs, psds[7])
#plt.plot(freqs2, psds2[7])

ImportError: Failed to import any of the following Qt binding modules: PyQt6, PySide6, PyQt5, PySide2

In [18]:
band_power_abs = {'d1': np.array([1,2,3,4]), 'd2': np.array([5,6,7,8])}
total_power = sum(band_power_abs.values())

# Test Report

In [13]:
from reportlab.lib.pagesizes import A4
from reportlab.lib import colors
from reportlab.platypus import SimpleDocTemplate, Paragraph, Table, TableStyle, Spacer
from reportlab.lib.styles import getSampleStyleSheet, ParagraphStyle
from reportlab.pdfbase.ttfonts import TTFont
from reportlab.pdfbase import pdfmetrics

# регистрируем кириллический шрифт
pdfmetrics.registerFont(TTFont("UI", str(FONT)))

doc = SimpleDocTemplate("report_reportlab.pdf", pagesize=A4,
                        leftMargin=24, rightMargin=24, topMargin=24, bottomMargin=24)

styles = getSampleStyleSheet()
styles.add(ParagraphStyle(name="TitleRU", fontName="UI", fontSize=20, leading=24, textColor=colors.white))
styles.add(ParagraphStyle(name="BodyRU", fontName="UI", fontSize=11, leading=14))

story = []

# цветная плашка-заголовок
from reportlab.platypus import Flowable
class Header(Flowable):
    def __init__(self, text, h=28, color=colors.HexColor("#1E5CB4")):
        Flowable.__init__(self); self.text=text; self.h=h; self.color=color
    def draw(self):
        c = self.canv
        c.saveState()
        c.setFillColor(self.color); c.rect(0, 0, self._width, self.h, fill=1, stroke=0)
        c.setFillColor(colors.white); c.setFont("UI", 18)
        c.drawString(10, 7, self.text)
        c.restoreState()
    def wrap(self, w, h): self._width=w; return w, self.h

story += [Header("Ежемесячный отчёт"), Spacer(1, 12)]
story += [Paragraph("Август 2025 • Отдел исследований", styles["BodyRU"]), Spacer(1, 8)]
story += [Paragraph("Краткое резюме: стабильность выросла, метрики улучшены, датасет расширен.", styles["BodyRU"]), Spacer(1, 12)]

data = [
    ["Метрика", "Значение", "Комментарий"],
    ["Accuracy", "0.92", "Норма > 0.90"],
    ["Precision", "0.89", "Рост на 0.02"],
    ["Recall", "0.87", "Стабильность выше"],
    ["F1-score", "0.88", "Баланс классов"],
]
table = Table(data, colWidths=[90, 70, 300])
table.setStyle(TableStyle([
    ("BACKGROUND", (0,0), (-1,0), colors.HexColor("#EAF0FE")),
    ("TEXTCOLOR", (0,0), (-1,0), colors.HexColor("#142030")),
    ("FONTNAME", (0,0), (-1,-1), "UI"),
    ("FONTSIZE", (0,0), (-1,0), 11),
    ("FONTSIZE", (0,1), (-1,-1), 10),
    ("ROWBACKGROUNDS", (0,1), (-1,-1), [colors.white, colors.HexColor("#FAFAFC")]),
    ("GRID", (0,0), (-1,-1), 0.3, colors.HexColor("#D0D0D0")),
    ("ALIGN", (1,1), (1,-1), "CENTER"),
]))
story += [table]

doc.build(story)
print("OK -> report_reportlab.pdf")


OK -> report_reportlab.pdf


In [30]:
from reportlab.lib.pagesizes import A4
from reportlab.lib import colors
from reportlab.platypus import SimpleDocTemplate, Paragraph, Table, TableStyle, Spacer
from reportlab.lib.styles import getSampleStyleSheet, ParagraphStyle
from reportlab.pdfbase.ttfonts import TTFont
from reportlab.pdfbase import pdfmetrics
from pathlib import Path

# 1) Найдём шрифт (укажи свой путь, если не найдётся)
FONT = next((Path(p) for p in [
    "/usr/share/fonts/truetype/dejavu/DejaVuSans.ttf",          # Linux
    "/System/Library/Fonts/Supplemental/Arial.ttf",             # macOS
    "/Library/Fonts/Arial.ttf",                                 # macOS
    "C:/Windows/Fonts/arial.ttf",                               # Windows
    "./DejaVuSans.ttf",
] if Path(p).exists()), None)
assert FONT and FONT.exists(), "Укажи путь к TTF-шрифту с кириллицей"

# Попробуем найти bold-начертание; если нет — используем обычный
BOLD = None
for cand in [
    str(FONT).replace(".ttf", "-Bold.ttf"),
    str(FONT).replace(".ttf", " Bold.ttf"),
    str(FONT).replace(".ttf", "Bold.ttf"),
]:
    if Path(cand).exists():
        BOLD = Path(cand); break
if BOLD is None:
    BOLD = FONT  # фолбэк

pdfmetrics.registerFont(TTFont("UI", str(FONT)))
pdfmetrics.registerFont(TTFont("UIB", str(BOLD)))

# 2) Стили
styles = getSampleStyleSheet()
styles.add(ParagraphStyle(name="H1RU", fontName="UIB", fontSize=12, leading=15, spaceAfter=6))
styles.add(ParagraphStyle(name="BodyRU", fontName="UI", fontSize=11, leading=14))
styles.add(ParagraphStyle(name="LblRU", fontName="UIB", fontSize=12, leading=14))
styles.add(ParagraphStyle(name="SmallRU", fontName="UI", fontSize=10, leading=12))

# 3) Документ
doc = SimpleDocTemplate(
    "eeg_report.pdf", pagesize=A4,
    leftMargin=36, rightMargin=36, topMargin=36, bottomMargin=36
)

story = []

# Поля «ФИО / Дата / Процедура» с линиями
def line_field(label, width=400):
    t = Table(
        [[Paragraph(label, styles["LblRU"]), ""]],
        colWidths=[85, width]
    )
    t.setStyle(TableStyle([
        ("FONT", (0,0), (0,0), "UIB", 12),
        ("BOTTOMPADDING", (1,0), (1,0), 2),
        ("LINEBELOW", (1,0), (1,0), 0.7, colors.black),
        ("VALIGN", (0,0), (-1,-1), "MIDDLE"),
    ]))
    return t

story += [line_field("ФИ:"), Spacer(1, 5)]
story += [line_field("Дата:"), Spacer(1, 10)]
story += [line_field("Процедура:"), Spacer(1, 14)]

# Разделы
story += [Paragraph("Результаты:", styles["H1RU"]), Spacer(1, 6)]
story += [Paragraph("Ритмы в фоновой активности до процедуры:", styles["H1RU"]), Spacer(1, 6)]

# Таблица метрик
headers = [
    "Область",
    "Дельта\n(абс.)", "Дельта /\nΣ",
    "Тета\n(абс.)",   "Тета /\nΣ",
    "Альфа\n(абс.)",  "Альфа /\nΣ",
    "Бета\n(абс.)",   "Бета /\nΣ",
]
rows = [
    ["Фронтальная", "", "", "", "", "", "", "", ""],
    ["Центральная (L)", "", "", "", "", "", "", "", ""],
    ["Центральная (R)", "", "", "", "", "", "", "", ""],
    ["Затылочная", "", "", "", "", "", "", "", ""],
]
data = [headers] + rows

tbl = Table(
    data,
    colWidths=[195, 48, 42, 48, 42, 48, 42, 48, 42],
    rowHeights=None,
    repeatRows=1,  # повторять заголовок на новых страницах
)

tbl.setStyle(TableStyle([
    # Шрифты
    ("FONT", (0,0), (-1,0), "UIB", 10),     # заголовок
    ("FONT", (0,1), (-1,-1), "UI", 10),     # тело
    # Выравнивание
    ("VALIGN", (0,0), (-1,-1), "MIDDLE"),
    ("ALIGN",  (1,1), (-1,-1), "CENTER"),
    ("ALIGN",  (0,0), (0,-1), "LEFT"),
    # Фон и сетка
    ("BACKGROUND", (0,0), (-1,0), colors.HexColor("#ECEFF7")),
    ("ROWBACKGROUNDS", (0,1), (-1,-1), [colors.white, colors.HexColor("#FAFAFC")]),
    ("GRID", (0,0), (-1,-1), 0.4, colors.HexColor("#C8CDD9")),
    # Визуальный разделитель под шапкой
    ("LINEBELOW", (0,0), (-1,0), 0.8, colors.HexColor("#AAB2C5")),
    # Внутренние отступы
    ("LEFTPADDING", (0,0), (-1,-1), 6),
    ("RIGHTPADDING", (0,0), (-1,-1), 6),
    ("TOPPADDING", (0,0), (-1,-1), 4),
    ("BOTTOMPADDING", (0,0), (-1,-1), 4),
]))
story += [tbl, Spacer(1, 14)]

# Сноска
story += [Paragraph("Σ — сумма абсолютных величин всех ритмов (Δ + Θ + α + β) для данной области.", styles["SmallRU"])]

doc.build(story)
print("OK → eeg_report.pdf")


OK → eeg_report.pdf


In [21]:
from fpdf import FPDF, XPos, YPos
import os

class PDF(FPDF):
    def colored_table(self, header, data):
        # Цвета и ширина линий
        self.set_fill_color(51, 122, 183)  # Синий цвет для шапки
        self.set_text_color(255, 255, 255)  # Белый текст для шапки
        self.set_draw_color(128, 128, 128)  # Серый цвет границ
        self.set_line_width(0.3)

        # Ширина колонок
        col_width = 45

        # Шапка таблицы
        self.set_font('Arial', 'B', 12)
        for col in header:
            self.cell(col_width, 10, text=col, border=1, align='C', fill=True)
        self.ln()

        # Данные таблицы
        self.set_fill_color(245, 245, 245)  # Светло-серый для строк
        self.set_text_color(0, 0, 0)  # Черный текст
        self.set_font('Arial', '', 10)

        fill = False
        for row in data:
            for item in row:
                self.cell(col_width, 8, text=str(item), border=1, align='C', fill=fill)
            self.ln()
            fill = not fill  # Чередование цвета строк

# Создаем PDF
pdf = PDF()
pdf.add_page()

# Добавляем системный шрифт
if os.name == 'nt':  # Windows
    font_path = 'C:/Windows/Fonts/arial.ttf'
    try:
        pdf.add_font('Arial', '', font_path)
        pdf.add_font('Arial', 'B', 'C:/Windows/Fonts/arialbd.ttf')  # Bold версия
    except:
        # Если не удается добавить Arial, используем встроенный шрифт
        pass
else:  # Mac/Linux
    try:
        if os.path.exists('/System/Library/Fonts/Supplemental/Arial.ttf'):
            # macOS
            pdf.add_font('Arial', '', '/System/Library/Fonts/Supplemental/Arial.ttf')
            pdf.add_font('Arial', 'B', '/System/Library/Fonts/Supplemental/Arial Bold.ttf')
        else:
            # Linux - попробуем другие пути
            pdf.add_font('Arial', '', '/usr/share/fonts/truetype/liberation/LiberationSans-Regular.ttf')
            pdf.add_font('Arial', 'B', '/usr/share/fonts/truetype/liberation/LiberationSans-Bold.ttf')
    except:
        # Используем встроенный шрифт, если не удается найти Arial
        pass

# Заголовок
pdf.set_font('Arial', 'B', size=16)
# Исправлено: используем text вместо txt, и new_x/new_y вместо ln
pdf.cell(200, 10, text='Привет Мир!', new_x=XPos.LMARGIN, new_y=YPos.NEXT, align='C')

# Добавляем отступ перед таблицей
pdf.ln(10)

# Заголовок таблицы
pdf.set_font('Arial', 'B', size=14)
pdf.cell(200, 10, text='Пример таблицы с данными', new_x=XPos.LMARGIN, new_y=YPos.NEXT, align='C')
pdf.ln(5)

# Данные для таблицы
header = ['Имя', 'Возраст', 'Город', 'Профессия']
data = [
    ['Иван', '25', 'Москва', 'Программист'],
    ['Мария', '30', 'Санкт-Петербург', 'Дизайнер'],
    ['Петр', '28', 'Новосибирск', 'Менеджер'],
    ['Анна', '35', 'Екатеринбург', 'Аналитик'],
    ['Дмитрий', '32', 'Казань', 'Инженер']
]

# Создаем таблицу
pdf.colored_table(header, data)

# Добавляем еще текст после таблицы
pdf.ln(1)
pdf.set_font('Arial', '', size=12)
pdf.set_text_color(0, 255, 0)
pdf.multi_cell(0, 10, text='Это пример PDF документа с таблицей, у которой синяя шапка. '
                           'Строки таблицы чередуются по цвету для лучшей читаемости.')

# Сохраняем PDF
pdf.output('отчет.pdf')
print("PDF файл 'отчет.pdf' успешно создан!")

PDF файл 'отчет.pdf' успешно создан!


In [41]:
def truncate(val, decimals=3):
    sig = max(0, 4 - len(str(int(val))))
    return format(val , '.%df' % sig)

val = 0.00000000012142
data = truncate(val, decimals=3)
print(data)

0.000
